[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AI-Hypercomputer/maxtext/blob/main/src/maxtext/examples/sft_qwen3_demo.ipynb)

# Qwen3-0.6B Supervised Fine-Tuning (SFT) Demo


## Overview

This notebook performs SFT training and evaluation workflow on [OpenAI's GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k).
The primary goal is to demonstrate the end-to-end process of:
1. Pre-SFT Evaluation: Calcuating baseline accuracy for the model before training.
2. SFT Training: Fine-tune the model using MaxText & Tunix SFT trainer.
3. Post-SFT Evaluation: Re-running the evaluation loop after training to measure the performance gain achieved by SFT.

This notebook can run on the **public TPU v5e-1**.

## Prerequisites

### Change Runtime Type (only if running on Google Colab)

**Instructions:**
1.  Navigate to the menu at the top of the screen.
2.  Click on **Runtime**.
3.  Select **Change runtime type** from the dropdown menu.
4.  Select **v5e-1 TPU** as the **Hardware accelerator**.
5. Click on **Save**.

### Get Your Hugging Face Token

To access model checkpoint from the Hugging Face Hub, you need to authenticate with a personal access token.

**Follow these steps to get your token:**

1.  **Navigate to the Access Tokens page** in your Hugging Face account settings. You can go there directly by visiting this URL:
    *   [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

2.  **Create a new token** by clicking the **"+ Create new token"** button.

3.  **Give your token a name** and assign it a **`read` role**. The `read` role is sufficient for downloading models.

4.  **Copy the generated token**. You will need this in the later steps.

**Follow these steps to store your token (only if running on Google Colab):**

1. On the left sidebar of your Colab window, click the key icon (the Secrets tab).

2. Click **"+ Add new secret"**.

3. Set the Name as **HF_TOKEN**.

4. Paste your token into the Value field.

5. Ensure the Notebook access toggle is turned On.

In [ ]:
try:
  from google.colab import userdata
  print("Running the notebook on Google Colab")
  IN_COLAB = True
except ImportError:
    print("Running the notebook on Visual Studio or JupyterLab")
    IN_COLAB = False

## Installation: MaxText & Other Dependencies

**⚠️ Note:** The installation process in following cell may take a few minutes to complete. Please be patient.

In [ ]:
try:
    import google.colab
    print("Running the notebook in Google Colab")
    IN_COLAB = True
except ImportError:
    print("Running the notebook on JupyterLab")
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    !git clone https://github.com/AI-Hypercomputer/maxtext.git
    %cd /content/maxtext

    # Install uv, a fast Python package installer
    !pip install uv

    # Install MaxText and its dependencies
    !uv pip install -e .[tpu] --resolution=lowest
    !python3 -m MaxText.install_maxtext_extra_deps

    # Install Tunix for post-training notebooks
    !uv pip install git+https://github.com/google/tunix
    
    # Install vllm for post-training notebooks
    !git clone https://github.com/vllm-project/vllm.git
    !VLLM_TARGET_DEVICE="tpu" uv pip install ./vllm

    # Install tpu-inference for post-training notebooks
    !git clone https://github.com/vllm-project/tpu-inference.git
    !uv pip install ./tpu-inference
    !uv pip install --no-deps qwix==0.1.4

### Restart Session (only if running on Google Colab)
To apply certain changes, you need to restart the session.

**Instructions:**
1.  Navigate to the menu at the top of the screen.
2.  Click on **Runtime**.
3.  Select **Restart session** from the dropdown menu.

You will be asked to confirm the action in a pop-up dialog. Click on **Yes**.

## Imports

In [ ]:
import jax
import os
import sys
import transformers

import MaxText
from maxtext.configs import pyconfig
from maxtext.examples.sft_train_and_evaluate import evaluate_model, get_test_dataset
from maxtext.integration.tunix.tunix_adapter import TunixMaxTextAdapter
from maxtext.utils.globals import MAXTEXT_REPO_ROOT, MAXTEXT_PKG_DIR
from maxtext.trainers.post_train.sft import train_sft

# Suppress vLLM logging with a severity level below ERROR
os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"
from tunix.rl.rollout import base_rollout
from tunix.rl.rollout.vllm_rollout import VllmRollout

from datetime import datetime
from flax import nnx
from huggingface_hub import login

print(f"MaxText installation path: {MAXTEXT_PKG_DIR}")

In [ ]:
if not jax.distributed.is_initialized():
  jax.distributed.initialize()
print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")

In [ ]:
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get("HF_TOKEN")
except ImportError:
    HF_TOKEN = os.environ.get("HF_TOKEN", "")

# If not found in the environment, prompt the user for input securely
# getpass function ensures the token is hidden while you type
if not HF_TOKEN:
    from getpass import getpass
    HF_TOKEN = getpass("Hugging Face token not found in environment. Please enter it here: ")

if HF_TOKEN:
    login(token=HF_TOKEN)
    print("Authenticated with Hugging Face successfully!")
else:
    print("Authentication failed: Hugging Face token is not set.")

## Model Configurations

In [ ]:
MODEL_NAME = "qwen3-0.6b"
TOKENIZER_PATH = "Qwen/Qwen3-0.6B"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    TOKENIZER_PATH,
    token=HF_TOKEN,
)

# set the path to the model checkpoint or leave empty to download from HuggingFace
MODEL_CHECKPOINT_PATH = ""
if not MODEL_CHECKPOINT_PATH:
   MODEL_CHECKPOINT_PATH = f"{MAXTEXT_PKG_DIR}/qwen_checkpoint"
   print("Model checkpoint will be downloaded from HuggingFace at: ",  MODEL_CHECKPOINT_PATH)
   print("Set MODEL_CHECKPOINT_PATH if you do not wish to download the checkpoint.")


RUN_NAME = datetime.now().strftime("%Y-%m-%d-%H-%m-%S")

# This is the directory where the fine-tuned model checkpoint will be saved
BASE_OUTPUT_DIRECTORY = f"{MAXTEXT_PKG_DIR}/maxtext_qwen06_output"

## Download Qwen3-0.6B Model Checkpoint from Hugging Face

In [ ]:
if not os.path.exists(MODEL_CHECKPOINT_PATH):
    # install torch for the conversion script
    !python3 -m pip install torch --index-url https://download.pytorch.org/whl/cpu

    !JAX_PLATFORMS=cpu PYTHONPATH={MAXTEXT_PKG_DIR} {sys.executable} -m maxtext.checkpoint_conversion.to_maxtext \
      {MAXTEXT_PKG_DIR}/configs/base.yml \
      model_name={MODEL_NAME} \
      base_output_directory={MODEL_CHECKPOINT_PATH} \
      hf_access_token={HF_TOKEN} \
      use_multimodal=false \
      scan_layers=true \
      skip_jax_distributed_system=True

if not os.path.exists(MODEL_CHECKPOINT_PATH):
    raise ValueError("Model checkpoint conversion failed. Check the logs above.")

## Dataset Configurations

In [ ]:
DATASET_NAME = "openai/gsm8k"
TRAIN_DATA_SPLIT = "train"
TEST_DATA_SPLIT = "test"
HF_DATA_DIR = "main"
TRAIN_DATA_COLUMNS = ["question", "answer"]
CHAT_TEMPLATE_PATH = f"{MAXTEXT_REPO_ROOT}/src/maxtext/examples/chat_templates/math_qa.json"
if not os.path.exists(CHAT_TEMPLATE_PATH):
    raise FileNotFoundError(f"Chat template not found: {CHAT_TEMPLATE_PATH}")
NUM_TEST_SAMPLES = 20  # Total number of samples to test
BATCH_SIZE = 1  # Number of test samples to process in a batch

## MaxText Configurations

In [ ]:
%%capture
config = pyconfig.initialize(
    [
        "",
        f"{MAXTEXT_PKG_DIR}/configs/post_train/sft.yml",
        f"load_parameters_path={MODEL_CHECKPOINT_PATH}/0/items",
        f"model_name={MODEL_NAME}",
        f"hf_access_token={HF_TOKEN}",
        f"base_output_directory={BASE_OUTPUT_DIRECTORY}",
        f"run_name={RUN_NAME}",
        f"tokenizer_path={TOKENIZER_PATH}",
        f"hf_path={DATASET_NAME}",
        f"train_split={TRAIN_DATA_SPLIT}",
        f"hf_data_dir={HF_DATA_DIR}",
        f"train_data_columns={TRAIN_DATA_COLUMNS}",
        "steps=500",
        "per_device_batch_size=1",
        "max_target_length=1024",
        "learning_rate=3e-6",
        "weight_dtype=bfloat16",
        "dtype=bfloat16",
        f"chat_template_path={CHAT_TEMPLATE_PATH}",
    ]
)

## Initial Setup & Data Preparation

### Create Test Dataset

In [ ]:
test_dataset = get_test_dataset(config, tokenizer)
test_dataset = test_dataset[:NUM_TEST_SAMPLES]
test_dataset = test_dataset.to_iter_dataset().batch(BATCH_SIZE, drop_remainder=True)
TOTAL_BATCHES = NUM_TEST_SAMPLES // BATCH_SIZE
print(
    f"Processing {NUM_TEST_SAMPLES} examples with a batch size of {BATCH_SIZE}. This will result in {TOTAL_BATCHES} total batches for the test run."
)

### Create SFT Trainer State

In [ ]:
trainer, mesh = train_sft.setup_trainer_state(config)

### Create vLLM Rollout

In [ ]:
tunix_model = TunixMaxTextAdapter(trainer.model)
vllm_rollout = VllmRollout(
    model=tunix_model,
    tokenizer=tokenizer,
    cache_config_or_size=1280,
    mesh=mesh,
    rollout_config=base_rollout.RolloutConfig(
        rollout_vllm_model_version=TOKENIZER_PATH,
        rollout_vllm_hbm_utilization=0.8,
        rollout_vllm_init_with_random_weights=True,
        rollout_vllm_tpu_backend_type="jax",
    ),
)

## Evaluation before SFT Training

In [ ]:
print("Running Pre-SFT Evaluation...")
score = evaluate_model(test_dataset, vllm_rollout, debug=False)

In [ ]:
print("========================= Score for PRE-SFT Evaluation =========================")
print(f"Percentage of test samples where the model produced the correct numerical answer: {score['correct']}%")
print(
    f"Percentage of test samples where the model produced the numerical answer within 10%: {score['partially_correct']}%"
)
print(
    f"Percentage of test samples where the model's output adheres to the expected structure: {score['correct_format']}%"
)

## SFT Training

In [ ]:
print("Starting SFT Training...")
trainer = train_sft.train_model(config, trainer, mesh)
print("SFT Training Complete!")

## Evaluation after SFT Training

In [ ]:
print("Running Post-SFT Evaluation...")
model = TunixMaxTextAdapter(trainer.model)
state = nnx.state(model)
vllm_rollout.update_params(state)
score = evaluate_model(test_dataset, vllm_rollout, debug=False)

In [ ]:
print("========================= Score for POST-SFT Evaluation =========================")
print(f"Percentage of test samples where the model produced the correct numerical answer: {score['correct']}%")
print(
    f"Percentage of test samples where the model produced the numerical answer within 10%: {score['partially_correct']}%"
)
print(
    f"Percentage of test samples where the model's output adheres to the expected structure: {score['correct_format']}%"
)